## Q1. Running Elastic

Run Elastic Search 8.17.6, and get the cluster information. What's the `version.build_hash` value?

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```

In [14]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()['version']['build_hash']

'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7'

## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Which function do you use for adding your data to elastic?

In [8]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

* `index`

## Q3. Searching

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

In [31]:
query_3 = "How do execute a command on a Kubernetes pod?"

search_query_3 = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_3,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response_3 = es_client.search(index=index_name, body=search_query)

In [32]:
response_3['hits']['hits'][0]['_score']

44.50556

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

In [64]:
query_4 = "How do copy a file to a Docker container?"

search_query_4 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_4,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response_4 = es_client.search(index=index_name, body=search_query_4)

In [65]:
response_4['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

In [66]:
result_docs_5 = []

for hit in response_4['hits']['hits']:
    result_docs_5.append(hit['_source'])

In [67]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc in result_docs_5:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query_4, context=context).strip()

What's the length of the resulting prompt? (use the `len` function)

In [70]:
len(prompt)

1446

## Q6. Tokens

Let's calculate the number of tokens in our query: 

In [73]:
import tiktoken

In [74]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [75]:
tokens = encoding.encode(prompt)

In [76]:
print(len(tokens))

320
